## Step 1: Reads in and parses the file of apple primary transcripts 

In [1]:
# Robert Bennett - 2/24/2020
# Programming 2 - BioPython Lab

from Bio import SeqIO
from Bio.Seq import Seq

# Creating list to hold records
dnaList = []
# Iterating through the file, parsing through records, and saving sequences >= 126 to dnaList
for record in SeqIO.parse("Mdomestica_491_v1.1.cds_primaryTranscriptOnly.fa", "fasta"):
    if len(record.seq) < 126:
        dnaList.append(record)

# Printing number of added transcripts
print("Number of transcripts: ", len(dnaList))

# Creating list to hold protein sequences
proteinList = []
# Iterating through dnaList and translating the seuqnces, adding them to proteinList
for seq in dnaList:
    proteinList.append(seq.translate())

Number of transcripts:  62


## Step 2: Aligns the small proteins using pairwise2
    

In [2]:
from Bio import pairwise2
from Bio.SubsMat.MatrixInfo import blosum62
import itertools

# Creating list to hold alignments
alignList = []
# Using itertools to interate through each unique comparison within proteinList
for a, b in itertools.combinations(proteinList, 2):
    # Aligning the unique indexes and saving them to a temp variable
    x = pairwise2.align.globalds(a.seq[0:-1], b.seq[0:-1], blosum62, -10, -0.5)
    # Checking to see if the score of x is greater than 40 and saving the alignment if it is
    if x[0][2] > 40:
        alignList.append(x)

# Iterating through the list and printing the formatted alignments saved from previous for loop
for list in alignList:
    for align in list:
        print(pairwise2.format_alignment(*align))

MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL
||||||||||||||||||||||||||||||||||||||||
MNESWVFTTVQHAISNPNGKRQSLKLQPRSLLQSCMGKPL
  Score=211



## Step 3: Runs BLAST and reads results

In [3]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# Choosing the protein to use
protein = proteinList[3]

# Getting the HSP from blast and reading the results
result = NCBIWWW.qblast("blastp", "nr", protein.seq)
record = NCBIXML.read(result)

In [4]:
# Setting the E value to parse with
E_VALUE_THRESH = 0.05
# Iterating through the alignments returned
for alignment in record.alignments:
    # Iterating through the high sequence pairs
    for hsp in alignment.hsps:
        # If the E value for the HSP is higher than the threshhold, print information about HSP
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

****Alignment****
sequence: gb|KAB2632446.1| S ribonuclease [Pyrus ussuriensis x Pyrus communis]
length: 808
e value: 0.00940533
LSAHGTSEIFSKNMGMFVRLCRSRWYIQTSHSSN...
+ A    E FSKNM + +RLCR+RWYIQ S  SN...
VGARAAVENFSKNMVVILRLCRARWYIQLSILSN...
